<a href="https://colab.research.google.com/github/GeoPythonGeek/Forest-Biomass-ML/blob/main/GEE-Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap

     |████████████████████████████████| 1.9 MB 8.3 MB/s 
     |████████████████████████████████| 130 kB 70.5 MB/s 
     |████████████████████████████████| 3.3 MB 64.8 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 8.5 MB 42.3 MB/s 
     |████████████████████████████████| 95 kB 4.0 MB/s 
     |████████████████████████████████| 1.3 MB 55.4 MB/s 
     |████████████████████████████████| 76 kB 6.3 MB/s 
     |████████████████████████████████| 1.2 MB 58.5 MB/s 
     |████████████████████████████████| 1.7 MB 59.4 MB/s 
     |████████████████████████████████| 216 kB 72.5 MB/s 
     |████████████████████████████████| 219 kB 72.8 MB/s 
     |████████████████████████████████| 98 kB 8.9 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 428 kB 76.3 MB/s 
     |████████████████████████████████| 395 kB 37.2 MB/s 
     |████████████████████████████████| 61 kB 8.6 MB/s 
     |█████████████████████

In [ ]:
!pip install geetools
#https://gis.stackexchange.com/questions/276500/google-earth-engine-export-an-entire-collection

     |████████████████████████████████| 74 kB 2.9 MB/s 
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=03328fcc20a63db9cf809a6cf8942e3bec1f7fd3e981c62e415f5598afe7eaa6
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built geetools


In [ ]:
import ee
import geemap
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=iiZEVmgIZuq9wblY_Rgr2qGBk7JORdgoQQmcrMdqy_U&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWg43xttbym0hK5yPzna6QaOE2KdsVLrMSq-n5a0CJnhThOrYOsvGks

Successfully saved authorization token.


In [ ]:
Map= geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
feature = Map.draw_last_feature
aoi = feature.geometry()

In [ ]:
!pip show geetools

Name: geetools
Version: 0.6.14
Summary: Set of tools to use in Google Earth Engine Python API
Home-page: https://github.com/gee-community/gee_tools
Author: Rodrigo E. Principe
Author-email: fitoprincipe82@gmail.com
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: pandas, pyshp, requests
Required-by: 


In [ ]:
#https://gis.stackexchange.com/questions/276500/google-earth-engine-export-an-entire-collection
import geetools
from geetools import batch
help(geetools.batch.Export.imagecollection.toDrive)

Help on function toDrive in module geetools.batch.imagecollection:

toDrive(collection, folder, namePattern='{id}', scale=30, dataType='float', region=None, datePattern=None, extra=None, verbose=False, **kwargs)
    Upload all images from one collection to Google Drive. You can use
    the same arguments as the original function
    ee.batch.export.image.toDrive
    
    :param collection: Collection to upload
    :type collection: ee.ImageCollection
    :param folder: Google Drive folder to export the images to
    :type folder: str
    :param namePattern: pattern for the name.
        See geetools.tools.image.make_name function and also
        geetools.tools.string.format function
    :type namePattern: str
    :param scale: scale of the image (side of one pixel). Defults to 30
        (Landsat resolution)
    :type scale: int
    :param dataType: as downloaded images **must** have the same data type
        in all bands, you have to set it here. Can be one of: "float",
        "dou

In [ ]:
#https://courses.spatialthoughts.com/end-to-end-gee.html #filtering-image-collections
#https://stackoverflow.com/questions/70589018/how-to-download-image-collection-from-google-earth-engine
collection = ee.ImageCollection('COPERNICUS/S2_SR') \
                .filter(ee.Filter.date('2021-03-01', '2021-04-28')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)) \
                .filter(ee.Filter.geometry(aoi))

In [ ]:
vis_params = {
    'min': 0.0,
    'max': 3000,
    'gamma': 1.3,
    'bands': ['B4','B3','B2']}

In [ ]:
mosaic = collection.mosaic().clip(aoi)

In [ ]:
#Masking images
Sentinel_bands = ["B2", "B3", "B4", "B5","B6","B7", "B8", "B9", "B11","B12","B8A"]

In [ ]:
sentinel = mosaic.select(Sentinel_bands)

In [ ]:
Map.addLayer(mosaic, vis_params, 'Mosaic')

In [ ]:
#Download the image to google drive
task = ee.batch.Export.image.toDrive(**{
    'image' : sentinel,
    'description' : 'Sentinel2',
    'folder' : 'export',
    'scale' : 20,
    'region' : aoi
})

task.start()

In [ ]:
task.status()

{'attempt': 1,
 'creation_timestamp_ms': 1642600203211,
 'description': 'Sentinel2',
 'id': 'XJUFYUO6CUOJATPAEEJNW5RG',
 'name': 'projects/earthengine-legacy/operations/XJUFYUO6CUOJATPAEEJNW5RG',
 'start_timestamp_ms': 1642600228788,
 'state': 'RUNNING',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1642601909627}

In [ ]:
#Calcualte NDVI google earth engine with python
#https://www.youtube.com/watch?v=Oc2BEfmOuNQ